In [24]:
import spacy
import random
import nltk
import tkinter as tk
from tkinter import scrolledtext, ttk
from nltk.sentiment import SentimentIntensityAnalyzer

# Load NLP models
nlp = spacy.load("en_core_web_sm")
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\2005s\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [25]:
# Movie database (Genres + Actors)
movie_db = {
    "action": ["Mad Max: Fury Road", "John Wick", "Die Hard"],
    "sci-fi": ["Interstellar", "Blade Runner 2049", "The Matrix"],
    "thriller": ["Gone Girl", "Seven", "Shutter Island"],
    "romance": ["Titanic", "The Notebook", "La La Land"],
    "Tom Cruise": ["Mission Impossible", "Top Gun", "Edge of Tomorrow"],
    "Leonardo DiCaprio": ["Inception", "The Revenant", "Titanic"]
}

In [26]:
# Extract movie genre or actor name
def extract_movie_info(user_input):
    user_input = user_input.lower()
    genre, actor = None, None
    
    # Check if input contains a genre or actor
    for key in movie_db.keys():
        if key.lower() in user_input:
            if key in ["Tom Cruise", "Leonardo DiCaprio"]:
                actor = key
            else:
                genre = key

    # Named Entity Recognition (NER) for actor names
    doc = nlp(user_input)
    for ent in doc.ents:
        if ent.label_ == "PERSON" and ent.text in movie_db:
            actor = ent.text

    return genre, actor


In [58]:
# Chatbot logic
def chatbot_response():
    user_input = user_entry.get()
    if not user_input:
        return

    chat_display.insert(tk.END, f"\nYou: {user_input}\n")
    user_entry.delete(0, tk.END)

    if user_input.lower() in ["exit", "quit", "stop"]:
        chat_display.insert(tk.END, "👋 Goodbye! Enjoy your movie!\n")
        return

    genre, actor = extract_movie_info(user_input)

    if genre:
        movie = random.choice(movie_db[genre])
        chat_display.insert(tk.END, f"🎥 MovieBot: I recommend watching '{movie}'\n")
    elif actor:
        movie = random.choice(movie_db[actor])
        chat_display.insert(tk.END, f"🎥 MovieBot: Since you like {actor}, try '{movie}'\n")
    else:
        chat_display.insert(tk.END, "🤔 MovieBot: Please mention a genre or an actor.\n")

    # Ask for feedback
    chat_display.insert(tk.END, "Did you like the suggestion? (yes/no)\n")
    feedback_button_frame.pack()

# Handle user feedback
def handle_feedback(response):
    sentiment = sia.polarity_scores(response)["compound"]
    if sentiment < 0:
        chat_display.insert(tk.END, "😔 MovieBot: Sorry! Let me suggest another movie.\n")
    else:
        chat_display.insert(tk.END, "🎉 MovieBot: Glad you liked it! Enjoy your movie!\n")
    feedback_button_frame.pack_forget()

# Create GUI using Tkinter
root = tk.Tk()
root.title("Movie Recommendation Chatbot")

# Chat display area
chat_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=15)
chat_display.pack(padx=10, pady=10)
chat_display.insert(tk.END, "🎬 Welcome to MovieBot! Tell me what kind of movies you like.\n")

# User input field
user_entry = tk.Entry(root, width=40)
user_entry.pack(padx=10, pady=5)

# Send button
send_button = tk.Button(root, text="Send", command=chatbot_response)
send_button.pack()

# Feedback buttons
feedback_button_frame = tk.Frame(root)
yes_button = tk.Button(feedback_button_frame, text="Yes 😊", command=lambda: handle_feedback("yes"))
no_button = tk.Button(feedback_button_frame, text="No 😞", command=lambda: handle_feedback("no"))
yes_button.pack(side=tk.LEFT, padx=5)
no_button.pack(side=tk.RIGHT, padx=5)


In [59]:
# Run GUI loop
root.mainloop()